In [2]:
from detr_model import DETR

preprocessor_config.json: 100%|██████████| 290/290 [00:00<00:00, 96.7kB/s]
`huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\heeji\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development


In [2]:
import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


: 

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

## DETR 모델 훈련, epoch=200

In [4]:
model = DETR()
train_data="./COCO_format/"
model.train(train_data,epochs=200)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


Some weights of DetrForObjectDetection were not initialized from the model checkpoint at facebook/detr-resnet-50 and are newly initialized because the shapes did not match:
- class_labels_classifier.weight: found shape torch.Size([92, 256]) in the checkpoint and torch.Size([5, 256]) in the model instantiated
- class_labels_classifier.bias: found shape torch.Size([92]) in the checkpoint and torch.Size([5]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name  | Type                   | Params
-------------------------------------------------
0 | model | DetrForObjectDetection | 41.5 M
-------------------------------------------------
41.3 M    Trainable params
222 K     Non-trainable params
41.5 M    Tot

Sanity Checking: 0it [00:00, ?it/s]

c:\Users\heeji\anaconda3\envs\yolo8\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:01<00:01,  1.22s/it]

c:\Users\heeji\anaconda3\envs\yolo8\lib\site-packages\pytorch_lightning\utilities\data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


c:\Users\heeji\anaconda3\envs\yolo8\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 199: 100%|██████████| 90/90 [00:25<00:00,  3.49it/s, v_num=3]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: 100%|██████████| 90/90 [00:30<00:00,  2.99it/s, v_num=3]


## 훈련한 모델 load

In [7]:
from transformers import DetrForObjectDetection
trained=DetrForObjectDetection.from_pretrained("detr_model").to(DEVICE)

In [138]:
model.predict("./COCO_format/test/left1009_jpg.rf.6d1f7225244659b4968d37642eef15a0.jpg")

Detections(xyxy=array([[     296.57,      174.35,       340.4,      358.55],
       [     295.84,      169.61,      340.96,      349.03],
       [     272.38,      108.55,      302.61,      188.26],
       [      292.3,      194.75,      327.11,      362.72],
       [     321.13,      237.58,      365.38,      415.84]], dtype=float32), mask=None, confidence=array([     0.9314,     0.72505,     0.99667,     0.99844,     0.99829], dtype=float32), class_id=array([3, 3, 1, 3, 3]), tracker_id=None)

## bounding box 표시하고 별도로 저장하기

In [20]:
def predict_and_save(img):
    import supervision as sv
    import cv2
    
    image = cv2.imread(img)
    print(img)
    classes = ["WaterTank","Water Tank","lift", "worker"]
    
    with torch.no_grad():
        detections = model.predict(img)
        labels = [f"{classes[class_id]} {confidence:0.2f}" for _, _, confidence, class_id, _ in detections]
        
    box_annotator = sv.BoxAnnotator()
    annotated_frame = box_annotator.annotate(scene=image.copy(), detections=detections, labels=labels)
    sv.plot_image(annotated_frame)
    
    parts = img.split("_jpg.rf.")
    parts = parts[0].split("/")[-1]
    output_path="./DETR_result/"+parts+".png"
    print(output_path)
    cv2.imwrite(output_path, annotated_frame)
    
    return

In [ ]:
import os
test_list=os.listdir('./COCO_format/test1')
    
for i in range(len(test_list)):
    predict_and_save('./test/images/'+test_list[i])

## map 구하기

### 훈련한 모델을 test 데이터에 적용하여 딕셔너리 형태로 supervision detection들을 저장

In [ ]:
test_list=os.listdir('./COCO_format/test1')
pred={}

for i in range(len(test_list)):
    
    parts = test_list[i].split("_jpg.rf.")
    parts = parts[0].split("/")[-1]
    img = './COCO_format/test1/'+test_list[i]
    pred[test_list[i]]=((model.predict(img)))


### 아래의 코드를 실행하여 ground_truth를 딕셔너리 형태로(dataset.annotations) 얻을 수 있음

In [ ]:
import supervision as sv
import numpy as np
from core_dataset import DetectionDataset

dataset = DetectionDataset.from_coco("./COCO_format/test1","./COCO_format/test/_annotations.coco.json")

### pred와 dataset.annotations의 정렬이 달라서 키 값을 기준으로 재정렬

In [201]:
sorted_keys1 = sorted(pred.keys())
print(sorted_keys1)

sorted_keys2 = sorted(dataset.annotations.keys())
print(sorted_keys2) 

pred = {key: pred[key] for key in sorted_keys1}
anno = {key: dataset.annotations[key] for key in sorted_keys2}


['left1009_jpg.rf.6d1f7225244659b4968d37642eef15a0.jpg', 'left1011_jpg.rf.80329927f937d116686bc3a6f913274a.jpg', 'left1014_jpg.rf.df67534395da28c5ebd381b83131efdd.jpg', 'left1038_jpg.rf.c92010b21ee8803474021aea5c6607b7.jpg', 'left1040_jpg.rf.63fc982a793fb0695a333bed41272fd2.jpg', 'left1047_jpg.rf.08068fca0ef4edf6c17ec9f14803c585.jpg', 'left1060_jpg.rf.19d990c1f9e55c5b644c702c6d6ffc82.jpg', 'left1077_jpg.rf.448534dd2757b590bf8c97023c04c48d.jpg', 'left1107_jpg.rf.34b2a89f4e1d3ea508927bbc5f126088.jpg', 'left1123_jpg.rf.e4144494a49e5a06a409ba150f475073.jpg', 'left1124_jpg.rf.d67db28622ad41ed2aa2dcff355806c6.jpg', 'left1136_jpg.rf.cb64a681b0e70e5a76e58c357b6573e0.jpg', 'left1138_jpg.rf.3afa657e912e7024e38d67ad2ff3332b.jpg', 'left1151_jpg.rf.7674c5e1bd7b8adcc196ef888742a903.jpg', 'left1152_jpg.rf.ca2b54f9023fa0025e8766a9026c96e5.jpg', 'left1274_jpg.rf.93315ad433438e1b9639677bd31666b5.jpg', 'left1302_jpg.rf.2b63af6730db97be9dee80e4cd879d86.jpg', 'left188_jpg.rf.1492c34da3cb9cb5b3c290f2755b005

In [213]:
print(pred.keys())
print(anno.keys())

dict_keys(['left1009_jpg.rf.6d1f7225244659b4968d37642eef15a0.jpg', 'left1011_jpg.rf.80329927f937d116686bc3a6f913274a.jpg', 'left1014_jpg.rf.df67534395da28c5ebd381b83131efdd.jpg', 'left1038_jpg.rf.c92010b21ee8803474021aea5c6607b7.jpg', 'left1040_jpg.rf.63fc982a793fb0695a333bed41272fd2.jpg', 'left1047_jpg.rf.08068fca0ef4edf6c17ec9f14803c585.jpg', 'left1060_jpg.rf.19d990c1f9e55c5b644c702c6d6ffc82.jpg', 'left1077_jpg.rf.448534dd2757b590bf8c97023c04c48d.jpg', 'left1107_jpg.rf.34b2a89f4e1d3ea508927bbc5f126088.jpg', 'left1123_jpg.rf.e4144494a49e5a06a409ba150f475073.jpg', 'left1124_jpg.rf.d67db28622ad41ed2aa2dcff355806c6.jpg', 'left1136_jpg.rf.cb64a681b0e70e5a76e58c357b6573e0.jpg', 'left1138_jpg.rf.3afa657e912e7024e38d67ad2ff3332b.jpg', 'left1151_jpg.rf.7674c5e1bd7b8adcc196ef888742a903.jpg', 'left1152_jpg.rf.ca2b54f9023fa0025e8766a9026c96e5.jpg', 'left1274_jpg.rf.93315ad433438e1b9639677bd31666b5.jpg', 'left1302_jpg.rf.2b63af6730db97be9dee80e4cd879d86.jpg', 'left188_jpg.rf.1492c34da3cb9cb5b3c29

### 재정렬된 딕셔너리들의 value값들을 MeanAveragePrecision 모듈에 입력하여 map 도출

In [212]:
def callback(image) -> sv.Detections:
    result = model.predict(image)
    return (result)

from detection import MeanAveragePrecision
mean_average_precision = MeanAveragePrecision.from_detections(pred.values(),anno.values())

print("map(50_95):", mean_average_precision.map50_95)
print("map(50):",mean_average_precision.map50)
print("map(75):",mean_average_precision.map75)
print("map(per_class):",mean_average_precision.per_class_ap50_95)

map(50_95): 0.6242863729761101
map(50): 0.9273284820217941
map(75): 0.6827641881872483
map(per_class): [[      0.978       0.978       0.978       0.978       0.978     0.92652     0.85585     0.71953     0.35999           0]
 [    0.87666     0.86565     0.81233     0.69797     0.64172     0.43901     0.23858     0.11132    0.044024   0.0065656]]


In [ ]:
#https://github.com/roboflow/supervision